## Cadences Network Visualization with Louvain Communities



In [3]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


# B.  Select Corpus

* The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
* The pieces are provided as a **list**, within square brackets and separated by commas.  
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: `corpus CorpusBase(
       ['https://crimproject.org/mei/CRIM_Mass_0006_1.mei',
       'https://crimproject.org/mei/CRIM_Mass_0006_2.mei',
       'https://crimproject.org/mei/CRIM_Mass_0006_3.mei'])`


In [4]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
                    'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_3.mei'
                    ,'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
                    'https://crimproject.org/mei/CRIM_Mass_0019_5.mei'])

Successfully imported https://crimproject.org/mei/CRIM_Model_0019.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_5.mei


In [5]:


list_of_dfs = corpus.batch(ImportedPiece.cadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone'] + "_" + combined_df['Evaded'].apply(str) + "_" + combined_df['Beat'].apply(str)
combined_df['some_features'] = combined_df['CadType'] + "_" + combined_df['Tone']
combined_df.copy
combined_df["shifted_all_feat"] = combined_df.groupby("Title")['all_features'].shift(-1)

### Network Visualization with Louvain Communities

* We first create a list of **successive cadences** in each piece.  These are **Cadence Pairs**.
* Each item in each pair records the following features:  **type**, **tone**, **evaded status**, and **beat strength of last note in the cadence**
* The pairs are thus highly distinct--consisting of four dimensions of two cadences each, such as: `('Clausula Vera_G_False_1.0', 'Clausula Vera_G_True_3.0')`
* This will create a varied network.

In [6]:
combined_df = combined_df.dropna(subset=['all_features', 'shifted_all_feat'])
cadence_pairs = list(zip(combined_df.all_features, combined_df.shifted_all_feat))

In [7]:
combined_df['all_features'].nunique()

32

### A Directed Graph of Pairs

* Directed graphs reflect the succession of events, and create a centrum of the most common 'goal' cadence.  Less frequent successions are at the edge of the graph

In [13]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black", directed=True)

G = nx.DiGraph()
# G.add_edges_from(unique_pairs)
G.add_edges_from(cadence_pairs)
# G = add_communities(G)
pyvis_graph.from_nx(G)
pyvis_graph.show('cadence_graph.html')